---

**Universidad de Costa Rica** | Escuela de Ingeniería Eléctrica

*IE0405 - Modelos Probabilísticos de Señales y Sistemas*

### `PyX` - Serie de tutoriales de Python para el análisis de datos

# `Py9` - *Simulaciones aleatorias*

> Conocidas como **simulaciones de Monte Carlo**, las simulaciones o experimentos aleatorios repiten una acción utilizando datos aleatorios para analizar sus resultados. El objetivo es conocer el comportamiento de un fenómeno o sistema para el cual se conoce el modelo de probabilidad de sus entradas y el modelo propio.

---

## Introducción

En contextos de sistemas de control o sistemas de comunicaciones es usual este tipo de simulaciones para probar estrategias de sintonización o esquemas de modulación digital.

---
## 8.1 - Simulaciones aleatorias aplicadas a transformaciones de variables aleatorias

Aun sin conocer una expresión para la función de densidad $f_Y(y)$ de una variable aleatoria $Y$ producto de una transformación $Y = T(X)$, es posible conocer el efecto que tiene esta transformación sobre una muestra de datos aleatorios con una distribución de probabilidad $X$ conocida.

### Ejemplo de una distribución normal con una transformación cuadrática

A una distribución $X \sim \mathcal{N}(0, 0.7737)$ se aplica una transformación $Y = X^2$. Es útil visualizar los dos conjuntos de datos.

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import csv

# Repeticiones
N = 500

# Distribución de X
Xva = stats.truncnorm(-2,2)

# Muestra de N valores aleatorios
X = Xva.rvs(N)

# Función aplicada sobre X
Y = X**2

# Crear histograma de X
a,b,c = plt.hist(X,20)
plt.show()

# Crear histograma de Y
a,b,c = plt.hist(Y,20)
plt.show()

---
## 8.2 - Ejemplo del canal de comunicaciones AWGN con la modulación OOK

La modulación **OOK** (*on/off keying*, codificación por encendido y apagado) consiste en enviar una forma de onda sinusoidal para el bit 1, y no enviar nada para el bit 0. Esta es una modulación binaria de un bit por símbolo.

#### Generación de los bits

In [ ]:
import numpy as np
from scipy import stats

# Número de bits
N = 10000

# Variable aleatoria binaria equiprobable
X = stats.bernoulli(0.5)

# Generar bits para "transmitir"
bits = X.rvs(N)

#### Generar la señal modulada OOK

Los parámetros son:

* Frecuencia de la portadora: $f =$ 1000 Hz
* Tiempo del símbolo: $T_s = T = 1/f$ (es decir, un período del símbolo es un período completo de la onda portadora)

In [ ]:
import matplotlib.pyplot as plt

# Frecuencia de operación
f = 1000 # Hz

# Duración del período de cada símbolo (onda)
T = 1/f # 1 ms

# Número de puntos de muestreo por período
p = 50

# Puntos de muestreo para cada período
tp = np.linspace(0, T, p)

# Creación de la forma de onda de la portadora
sinus = np.sin(2*np.pi * f * tp)

# Visualización de la forma de onda de la portadora
plt.plot(tp, sinus)
plt.xlabel('Tiempo / s')
plt.show()

# Frecuencia de muestreo
fs = p/T # 50 kHz

# Creación de la línea temporal para toda la señal Tx
t = np.linspace(0, N*T, N*p)

# Inicializar el vector de la señal modulada Tx
senal = np.zeros(t.shape)

# Creación de la señal modulada OOK
for k, b in enumerate(bits):
    senal[k*p:(k+1)*p] = b * sinus

# Visualización de los primeros bits modulados
pb = 5
plt.figure()
plt.plot(senal[0:pb*p])
plt.show()

#### Cálculo de la potencia promedio

Es conocido que:

$$
P(T) = \frac{1}{2T}\int_{-T}^{T}x^2(t) ~\mathrm{d}t = A\{x^2(t)\}
$$

y también que:

$$
P(T) = \frac{1}{2T}\int_{-T}^{T}E[X^2(t)] ~\mathrm{d}t = A\{E[X^2(t)]\}
$$

donde $A\{\cdot\}$ es el operador de promedio temporal y $E[X^2(t)]$ es la autocorrelación del proceso aleatorio $X(t)$.

In [ ]:
from scipy import integrate

# Potencia instantánea
Pinst = senal**2

# Potencia promedio a partir de la potencia instantánea (W)
Ps = integrate.trapz(Pinst, t) / (N * T)

#### Simulación de un canal ruidoso AWGN

Conociendo que la relación señal-a-ruido es

$$
SNR_{dB} = 10 \log_{10} \left( \frac{P_s}{P_n} \right)
$$

donde $P_s$ es la potencia de la señal y $P_n$ es la potencia del ruido ($n$ de *noise*).

In [ ]:
# Relación señal-a-ruido deseada
SNR = 3

# Potencia del ruido para SNR y potencia de la señal dadas
Pn = Ps / (10**(SNR / 10))

# Desviación estándar del ruido
sigma = np.sqrt(Pn)

# Crear ruido (Pn = sigma^2)
ruido = np.random.normal(0, sigma, senal.shape)

# Simular "el canal": señal recibida
Rx = senal + ruido

# Visualización de los primeros bits recibidos
pb = 5
plt.figure()
plt.plot(Rx[0:pb*p])
plt.show()

#### Densidad espectral de potencia

In [ ]:
from scipy import signal

# Antes del canal ruidoso
fw, PSD = signal.welch(senal, fs, nperseg=1024)
plt.figure()
plt.semilogy(fw, PSD)
plt.xlabel('Frecuencia / Hz')
plt.ylabel('Densidad espectral de potencia / V**2/Hz')
plt.show()

# Después del canal ruidoso
fw, PSD = signal.welch(Rx, fs, nperseg=1024)
plt.figure()
plt.semilogy(fw, PSD)
plt.xlabel('Frecuencia / Hz')
plt.ylabel('Densidad espectral de potencia / V**2/Hz')
plt.show()

#### Demodulación y decodificación de la señal

Es útil aquí el conocimiento del **producto interno** de dos señales, utilizado frecuentemente para modulación *coherente* (donde la fase, es decir, la sincronización, es perfecta). Se denota como:

$$
\langle g(t), h(t) \rangle = \int_0^T g(t) h(t) ~\mathrm{d}t
$$

Y aplica que:

* $\langle g(t), g(t) \rangle = E_g$, la energía de la señal en el intervalo de integración.
* $\langle g(t), h(t) \rangle = 0$ cuando $g(t)$ y $h(t)$ son ortogonales.

In [ ]:
# Pseudo-energía de la onda original (esta es suma, no integral)
Es = np.sum(sinus**2)

# Inicialización del vector de bits recibidos
bitsRx = np.zeros(bits.shape)

# Decodificación de la señal por detección de energía
for k, b in enumerate(bits):
    Ep = np.sum(Rx[k*p:(k+1)*p] * sinus)
    if Ep > Es/2:
        bitsRx[k] = 1
    else:
        bitsRx[k] = 0

err = np.sum(np.abs(bits - bitsRx))
BER = err/N

print('Hay un total de {} errores en {} bits para una tasa de error de {}.'.format(err, N, BER))

## 8.3 - Ejemplo de la aproximación del número $\pi$

(Pendiente)

## 8.4 - Ejemplo de un proceso de Markov

Con una tasa de llegada $\lambda$ y un parámetro de tiempo de servicio $\nu$, un proceso de Markov con un solo servidor también recibe la notación de *sistema de colas* **M/M/1** (donde la **M** viene de *Markov*). Más en general, con $s$ servidores es un sistema **M/M/s**.

La simulación de un sistema del tipo **M/M/1** implica la generación de una llegada de "clientes" como una *corriente de Poisson*. Esto es equivalente a decir que tienen una distribución de probabilidad de *tiempo entre arribos* con distribución exponencial y parámetro $\lambda$.

Por su parte, el *tiempo de servicio* tiene también una distribución exponencial pero con parámetro $\nu$. A la relación $\lambda/\nu$ usualmente se le conoce como $\rho$.

**Nota**: El tiempo de servicio se asume independiente del tiempo de llegada.

#### Sobre la simulación

Es posible crear una simulación de $N$ clientes con sus respectivos tiempos de servicio, distribuidos en el tiempo.

Para medir el tiempo se puede utilizar una medida mínima arbitraria, pero que tenga sentido para el problema. Por ejemplo: si la tasa de llegada es de 1 persona/minuto, vale más tener una granularidad de segundos o decenas de segundos para capturar una precisión temporal adecuada.

**Nota**: La selección de $N$ puede depender de la precisión deseada para el resultado. Por ejemplo: con $N = 1000$ es posible obtener una precisión de hasta el 0,1% ($1/N$).

### Problema de ejemplo: un servidor web

> Un servidor web es modelado como un sistema M/M/1 con una tasa de arribo de 2 solicitudes por minuto. Es deseado tener 4 o menos solicitudes en fila el 99\% del tiempo. ¿Qué tan rápido debe ser el servicio? $\nu$ es solicitudes atendidas por minuto.

El estado $i$ es el número de clientes en el sistema. La longitud de la fila es $L_q = i - 1$ (en virtud de la solicitud que está siendo atendida en $s = 1$ servidores). Es posible encontrar que:

$$
P( \text{5 o más clientes en el sistema} ) = \sum_{i=5}^{\infty} (1 - \rho) \rho^i  = 1 - \sum_{i=0}^{4} (1 - \rho) \rho^i = \rho^5
$$

que depende de $\rho = \lambda/\nu$ y del parámetro de servicio $\nu$ buscado. 

De los datos del problema: $\lambda = 2$. Para tener una fila de 3 o menos clientes el 99\% del tiempo se necesita:

$$
\begin{aligned}
P( \text{5 o más clientes en el sistema} ) = \rho^5 & = \left( \frac{\lambda}{\nu} \right)^5 \leq 0.01 \\
\nu^5 & \geq \frac{\lambda^5}{0.01} = \frac{2^5}{0.01} = 3200 \quad \Rightarrow \quad \nu \geq 5.024
\end{aligned}
$$

es decir, el servidor debe atender más de 5,024 solicitudes por minuto en promedio para poder satisfacer el requisito.

En la siguiente simulación, con $N = 1000$, y $\nu = 3 < 5.024$ deberíamos obtener una probabilidad $P( \text{5 o más clientes en el sistema} ) > 0.01$ que **no** cumple con las especificaciones.

**Nota**: Observar el cambio de unidades de minutos a segundos, para lograr mayor "granularidad".

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# Número de clientes
N = 1000

# Parámetro de llegada (clientes/segundos)
lam = 2/60

# Parámetro de servicio (servicios/segundos)
nu = 3/60

# Distribución de los tiempos de llegada entre cada cliente
X = stats.expon(scale = 1/lam)

# Distribución de los tiempos de servicio a cada cliente
Y = stats.expon(scale = 1/nu)

# Intervalos entre llegadas (segundos desde último cliente)
t_inte = np.ceil(X.rvs(N)).astype('int')

# Tiempos de las llegadas (segundos desde el inicio)
t_lleg = [t_inte[0]]
for i in range(1, len(t_inte)):
    siguiente = t_lleg[i-1] + t_inte[i]
    t_lleg.append(siguiente)

# Tiempos de servicio (segundos desde inicio de servicio)
t_serv = np.ceil(Y.rvs(N)).astype('int')

# Inicialización del tiempo de inicio y fin de atención
inicio = t_lleg[0]          # primera llegada
fin = inicio + t_serv[0]    # primera salida

# Tiempos en que recibe atención cada i-ésimo cliente (!= que llega)
t_aten = [inicio]
for i in range(1, N):
    inicio = np.max((t_lleg[i], fin))
    fin = inicio + t_serv[i]
    t_aten.append(inicio)

# Inicialización del vector temporal para registrar eventos
t = np.zeros(t_aten[-1] + t_serv[-1] + 1)

# Asignación de eventos de llegada (+1) y salida (-1) de clientes
for c in range(N):
    i = t_lleg[c]
    t[i] += 1
    j = t_aten[c] + t_serv[c]
    t[j] -= 1

# Umbral de P o más personas en sistema (hay P - 1 en fila)
P = 5

# Instantes (segundos) de tiempo con P o más solicitudes en sistema
frecuencia = 0

# Proceso aleatorio (estados n = {0, 1, 2...})
Xt = np.zeros(t.shape)

# Inicialización de estado n
n = 0

# Recorrido del vector temporal y conteo de clientes (estado n)
for i, c in enumerate(t):
    n += c # sumar (+1) o restar (-1) al estado
    Xt[i] = n
    if Xt[i] >= P: 
        frecuencia += 1

# Fracción de tiempo con P o más solicitudes en sistema
fraccion = frecuencia / len(t)

# Resultados
print('Parámetro lambda = ', str(lam*60))
print('Parámetro nu = ', str(nu*60))
print('Tiempo con más de {} solicitudes en fila:'.format(P-2))
print('\t {:0.2f}%'.format(100*fraccion))
if fraccion <= 0.01:
    print('\t Sí cumple con la especificación.')
else:
    print('\t No cumple con la especificación.') 
print('Simulación es equivalente a {:0.2f} horas.'.format(len(t)/3600))

# Gráfica de X(t) (estados del sistema)
plt.figure()
plt.plot(Xt)
plt.plot(range(len(t)), (P-1)*np.ones(t.shape))
plt.legend(('$X(t) = n$', '$L_q = $' + str(P-2)))
plt.ylabel('Clientes en el sistema, $n$')
plt.xlabel('Tiempo, $t$ / segundos')
plt.show()

---
### Más información

* Función [scipy.integrate.trapz](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.trapz.html)
* Modulación [OOK](https://en.wikipedia.org/wiki/On%E2%80%93off_keying)
---

---
**Universidad de Costa Rica** | Facultad de Ingeniería | Escuela de Ingeniería Eléctrica

&copy; 2022

---